In [1]:
# Khai báo thư viện
import numpy as np
from scipy.linalg import svd

In [2]:
# Các hàm sử dụng trong bài
def compute_svd (A):
    """
    Hàm tìm phân tích suy biến svd của một ma trận
    """
    # Dùng hàm svd để tính ma trận U, S_diag và Vt
    U, S_diag, Vt = svd (A)
    
    # Tạo ma trận không cấp k × k
    k = len(S_diag)
    matrix_rank_k = np.zeros((k, k))
    
    # Thay đường chéo của ma trận không bằng các phần tử của S_diag
    np.fill_diagonal(matrix_rank_k, S_diag)
    
    # Thêm hàng/cột 0 vào Σ để cùng dạng với ma trận A ban đầu
    Σ = np.zeros(A.shape)
    Σ[:k, :k] = matrix_rank_k
    
    return U, Σ, Vt

def get_svd_size_k(U, S_diag, Vt, k):
    """
    Hàm tính toán Uk, Σk, VkT từ ma trận A và giữ lại k hàng/cột đầu tiên
    """
    # Giữ lại k hàng và k cột đầu tiên của ma trận Σ để tạo ra Σk
    Σk = np.diag(S_diag[:k])
    
    # Giữ lại k cột đầu tiên của ma trận U để tạo ra Uk
    Uk = U[:, :k]
    
    # 5. Giữ lại k hàng đầu tiên của ma trận Vt để tạo ra VkT
    VkT = Vt[:k, :]
    
    return Uk, Σk, VkT

def compute_compact_svd(A):
    """
    Hàm tìm compact SVD của một ma trận
    """
    # Tìm SVD của ma trận A (dùng scipy)
    U, S_diag, Vt = svd(A, full_matrices=False)

    # Tìm số giá trị λ khác 0 của ma trận A, đặt là r
    r = np.count_nonzero(S_diag)
    
    # Tạo ma trận Ur, Σr, VrT
    Ur, Σr, VrT = get_svd_size_k(U, S_diag, Vt, r)
    
    return Ur, Σr, VrT

def compute_truncated_svd_know_k(A, k=4):
    """
    Hàm tìm truncated SVD khi biết số λ cần giữ lại, mặc định k=4
    """
    U, S_diag, Vt = svd(A, full_matrices=False)
    Uk, Σk, VkT = get_svd_size_k(U, S_diag, Vt, k)
    
    # Phần trăm thông tin còn giữ lại
    total_information = np.sum(S_diag**2)
    retained_information = np.sum(S_diag[:k]**2)
    I = (retained_information / total_information) * 100
    
    return Uk, Σk, VkT, I

def compute_truncated_svd_know_i (A, i=0.9):
    """
    Hàm tìm truncated SVD khi biết phần trăm lượng thông tin muốn giữ, 
    mặc định tỷ lệ thông tin cần giữ lại là 90%
    
    """
    U, S_diag, Vt = svd(A, full_matrices=False)

    # Tìm số lượng giá trị kì dị cần giữ lại để đạt được ngưỡng thông tin
    total_variance = np.sum(S_diag**2)
    cumulative_variance = 0
    for k in range(len(S_diag)):
        cumulative_variance += S_diag[k]**2
        if (cumulative_variance / total_variance) >= i:
            break

    Uk, Σk, VkT = get_svd_size_k(U, S_diag, Vt, k+1)

    return Uk, Σk, VkT, k+1

In [3]:
# Bài tập 1 - Tìm phân tích suy biến svd của một ma trận
A = np.array([[1, 0, 0, 0, 2], 
              [0, 0, 3, 0, 0], 
              [0, 0, 0, 0, 0],
              [0, 2, 0, 0, 0]])

U, Σ, Vt = compute_svd(A)
print(f'Ma trận U:')
print (U)

print (f'Ma trận Σ:')
print (Σ)

print (f'Ma trận Vt:')
print (Vt)

Ma trận U:
[[ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0. -1.  0.]]
Ma trận Σ:
[[3.         0.         0.         0.         0.        ]
 [0.         2.23606798 0.         0.         0.        ]
 [0.         0.         2.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]]
Ma trận Vt:
[[-0.          0.          1.         -0.          0.        ]
 [ 0.4472136   0.          0.          0.          0.89442719]
 [ 0.         -1.          0.          0.          0.        ]
 [ 0.          0.          0.          1.          0.        ]
 [-0.89442719  0.          0.          0.          0.4472136 ]]


In [4]:
# Bài tập 2 - Tìm compact SVD của một ma trận
A = np.array([[1, 0, 0, 0, 2], 
              [0, 0, 3, 0, 0], 
              [0, 0, 0, 0, 0],
              [0, 2, 0, 0, 0]])

Ur, Σr, VrT = compute_compact_svd(A)

print(f'Ma trận Ur:')
print (Ur)

print (f'Ma trận Σr:')
print (Σr)

print (f'Ma trận VrT:')
print (VrT)

Ma trận Ur:
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -1.]]
Ma trận Σr:
[[3.         0.         0.        ]
 [0.         2.23606798 0.        ]
 [0.         0.         2.        ]]
Ma trận VrT:
[[-0.          0.          1.         -0.          0.        ]
 [ 0.4472136   0.          0.          0.          0.89442719]
 [ 0.         -1.          0.          0.          0.        ]]


In [7]:
# Bài tập 3 - Tìm truncated SVD khi biết số λ cần giữ lại
A = np.array([[1.01, 0.9, 0.2, 1.001, 0.3], 
              [0.2, 1.01, 0.3, 0.8, 0.4], 
              [1, 1.002, 2, 0.98, 2],
              [0.3, 2, 0.4, 1.01, 0.9],
              [1.1, 0.2, 0.03, 2, 0.87]])
k=4
Uk, Σk, VkT, I = compute_truncated_svd_know_k (A, k)
print(f'Ma trận Uk:')
print(Uk)

print(f'Ma trận Σk:')
print(Σk)
    
print(f'Ma trận VkT:')
print(VkT)
    
print(f'Phần trăm lượng thông tin khi giữ lại {k} giá trị kì dị: {I:.2f}%')


Ma trận Uk:
[[-0.32813554  0.32928789  0.08780746 -0.88098834]
 [-0.27374756  0.09123211  0.31504405  0.16061502]
 [-0.64816455 -0.65873014 -0.3795124  -0.04270121]
 [-0.46609071  0.0207275   0.71918456  0.25646457]
 [-0.4243012   0.66999341 -0.4814364   0.36119923]]
Ma trận Σk:
[[4.68190663 0.         0.         0.        ]
 [0.         1.79325215 0.         0.        ]
 [0.         0.         1.57960998 0.        ]
 [0.         0.         0.         0.54903356]]
Ma trận VkT:
[[-0.35047463 -0.4780765  -0.35097791 -0.5344013  -0.48973477]
 [ 0.24274782 -0.05358443 -0.66685673  0.62343097 -0.32378718]
 [-0.3428965   0.86035897 -0.23658964 -0.1699711  -0.2394569 ]
 [-0.77612359 -0.16080247 -0.18212689  0.3391472   0.47284639]]
Phần trăm lượng thông tin khi giữ lại 4 giá trị kì dị: 99.89%


In [8]:
# Bài tập 4:
A = np.array([[1.01, 0.9, 0.2, 1.001, 0.3], 
              [0.2, 1.01, 0.3, 0.8, 0.4], 
              [1, 1.002, 2, 0.98, 2],
              [0.3, 2, 0.4, 1.01, 0.9],
              [1.1, 0.2, 0.03, 2, 0.87]])

i=0.9
Uk, Σk, VkT, k = compute_truncated_svd_know_i (A, i)

print(f'Ma trận Uk:')
print(Uk)

print(f'Ma trận Σk:')
print(Σk)
    
print(f'Ma trận VkT:')
print(VkT)

print(f'Số giá trị k cần giữ là: {k}')

Ma trận Uk:
[[-0.32813554  0.32928789  0.08780746]
 [-0.27374756  0.09123211  0.31504405]
 [-0.64816455 -0.65873014 -0.3795124 ]
 [-0.46609071  0.0207275   0.71918456]
 [-0.4243012   0.66999341 -0.4814364 ]]
Ma trận Σk:
[[4.68190663 0.         0.        ]
 [0.         1.79325215 0.        ]
 [0.         0.         1.57960998]]
Ma trận VkT:
[[-0.35047463 -0.4780765  -0.35097791 -0.5344013  -0.48973477]
 [ 0.24274782 -0.05358443 -0.66685673  0.62343097 -0.32378718]
 [-0.3428965   0.86035897 -0.23658964 -0.1699711  -0.2394569 ]]
Số giá trị k cần giữ là: 3
